In [ ]:
# Load in packages
## From FaceWord, ICA, stats
import pip
%pip install mne
!python -m pip install mne
import mne
!python -m pip install pandas
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
os.system('python -m pip install scikit-learn')
from scipy import stats as st
import statistics as stats
from mpl_toolkits.axes_grid1 import make_axes_locatable
!pip install scikit-learn
import sklearn # scikit-learn is needed to run ICA (Independent Component Analysis)
from scipy import stats
!pip install scikit-posthocs
import scikit_posthocs as sp

: 

In [ ]:
# Loading the data
file_path = "/Users/patrikmolnar/Downloads/own_group10.vhdr"
raw = mne.io.read_raw_brainvision(file_path,eog=('EOG1', 'EOG2'))
raw.load_data()

: 

In [ ]:
# Specify electrode locations
montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage, verbose=False)

: 

In [ ]:
# Electrode layout
layout_from_raw = mne.channels.make_eeg_layout(raw.info)
layout_from_raw.plot();

: 

In [ ]:
#Channel types
chans = mne.pick_types(raw.info, meg=False, eeg=True, eog=True)
chan_types = [mne.io.pick.channel_type(raw.info, ch) for ch in chans]
chan_names = raw.info['ch_names']
print('Channels are of type')
print(*('{}: {}'.format(ch, typ) for ch, typ in zip(chan_names, chan_types)), sep='\n')




: 

In [ ]:
# Plotting raw data from only EEG and stimulus data 
raw.pick_types(meg=False, eeg=True, eog=True, stim=True, exclude=[])
raw.plot(scalings=dict(eeg=20e-5));

# Plotting raw data without eog
raw.pick_types(meg=False, eeg=True, stim=True, exclude=[])
raw.plot(); # the ; at the end of the line suppresses the output (if not used, the plot will be displayed twice)

# Building the plot
## Plotting all EEG channels simultaneously
raw.plot(n_channels=60);

## Plotting a full minute from the recording
raw.plot(n_channels=60, duration=60);

: 

In [ ]:
#List and exclude the bad channels
raw.info['bads'] = ['T8']
raw.plot(n_channels=60);

# Remove/exclude bad channels
raw.pick_types(meg=False, eeg=True, stim=True, exclude='bads')
raw.plot(n_channels=60);

: 

In [ ]:
# Setting the average reference and plot
raw.set_eeg_reference(ref_channels='average', ch_type='eeg')

#high-pass filter
raw= raw.filter(0.1, None)

# low-pass
raw = raw.filter(None, 40)

raw.plot(n_channels=60);

: 

In [ ]:
# Set up and fit ICA with mne.preprocessing.ICA() 

## n_components=0.95 ensures that the number of components selected explain at least 95% of the variance in the data

ica = mne.preprocessing.ICA(n_components=0.95, random_state=7, max_iter=800)
ica.fit(raw)

: 

In [ ]:
# Plotting ICA components

ica.plot_components();

: 

In [ ]:
#Plotting ICA component time sources
ica.plot_sources(raw);

# blinks
ica.plot_overlay(raw, exclude=[0], picks="eeg");
# blinks
ica.plot_overlay(raw, exclude=[1], picks="eeg");

: 

In [ ]:
# Exclude eye noise and apply ICA

ica.exclude = [0, 1] # setting the exclude argument to the noise components
ica.plot_properties(raw, picks=ica.exclude); # plotting the components that will be excluded (sanity check, did we index the right ones?)

: 

In [ ]:
# Creating new data
raw_clean = ica.apply(raw.copy(), exclude=ica.exclude)

# Visualize the cleaned data
raw_clean.plot(n_channels=60);




: 

In [ ]:
#Data processing
raw_clean = raw_clean.filter(0.1, None)

# low-pass
raw_clean = raw_clean.filter(None, 40)
#rejecting non-brain activity
reject = {'eeg': 150e-6}
## For us, an "event" should be the presentation of a new stimuli, categorized as one of four levels
events = mne.events_from_annotations(raw_clean)[0]
events[:, 2]
## **Create a dictionary of what each event ID represents.**
### By using '/' we can actually later index one dimension *across* the other, i.e. if we just write 'left' we get all events presented to the left side, both auditory and visual
event_id = {'objects':11,
            'faces': 21,
            'pareidolias': 31,
            'uknown':100,
            'unknow':200
           }

tmin=-0.2 

tmax = 0.5

picks = mne.pick_types(raw_clean.info, meg=False, eeg=True, eog=False)

epochs = mne.Epochs(raw_clean, events, event_id, tmin, tmax, picks=picks,
                    baseline=(None, 0), reject=reject, preload=True)

epochs_resampled = epochs.resample(250)

#obj = epochs['objects']
#faces = epochs['faces']
#par = epochs['pareidolias']


#face_epochs = epochs_resampled['faces']
#obj_epochs = epochs_resampled['objects']
#par_epochs = epochs_resampled['pareidolias']

obj = epochs_resampled['objects']
faces = epochs_resampled['faces']
par = epochs_resampled['pareidolias']

par_evoked = par.average()

faces_evoked = faces.average()

obj_evoked = obj.average()


mne.viz.plot_events(events, sfreq=250, first_samp=raw_clean.first_samp, event_id=event_id);

#Joint Butterfly
par_evoked.plot_joint(picks='eeg');
faces_evoked.plot_joint(picks='eeg');
obj_evoked.plot_joint(picks='eeg');

#Topography
obj_evoked.plot_topomap();
par_evoked.plot_topomap();
faces_evoked.plot_topomap();

#visualization
raw_clean.plot_sensors(ch_type='eeg', show_names=True); # show_names=True shows the channel names
raw_clean.plot_sensors(kind='3d', ch_type='eeg'); # for a 3D view

: 

In [ ]:
#Analysis of N170
obj_data = obj.get_data(picks=['P3', 'P4', 'P7', 'P8'], tmin=.14, tmax=.22)
#print(obj_data)

faces_data = faces.get_data(picks=['P3', 'P4', 'P7', 'P8'], tmin=.14, tmax=.22)
#print(faces_data)

par_data = par.get_data(picks=['P3', 'P4', 'P7', 'P8'], tmin=.14, tmax=.22)
#print(par_data)

# Objects
obj_data_mean = np.mean(obj_data, axis=2) 
obj_data_mean = np.mean(obj_data_mean, axis=1) 


# Faces
faces_data_mean = np.mean(faces_data, axis=2)
faces_data_mean = np.mean(faces_data_mean, axis=1)


# Pareidolias
par_data_mean = np.mean(par_data, axis=2)
par_data_mean = np.mean(par_data_mean, axis=1)

# Normality testing

##Faces
ks_test = stats.kstest(faces_data_mean,'norm')
ks_test_statistic, ks_test_pvalue = ks_test
print(f"KS test statistic: {ks_test_statistic}, p-value: {ks_test_pvalue}")

##Parreidolias
ks_test = stats.kstest(par_data_mean, 'norm')
ks_test_statistic, ks_test_pvalue = ks_test
print(f"KS test statistic: {ks_test_statistic}, p-value: {ks_test_pvalue}")

##Objects
ks_test = stats.kstest(obj_data_mean, 'norm')
ks_test_statistic, ks_test_pvalue = ks_test
print(f"KS test statistic: {ks_test_statistic}, p-value: {ks_test_pvalue}")







# One-way (#ANOVA) t-test between the 2(3) means.

#st.f_oneway(faces_data_mean, par_data_mean, obj_data_mean)
#st.ttest_ind(faces_data_mean, obj_data_mean)

#t_stat, p_value = stats.ttest_ind(faces_data_mean, par_data_mean)

# Define the number of tests conducted
#num_tests = 2  # In this case, you have performed two independent t-tests

# Apply Bonferroni correction to the p-value
#corrected_p_value = p_value * num_tests


# Print the original and corrected p-values
#print(t_stat)
#print("Original p-value:", p_value)
#print("Corrected p-value (Bonferroni):", corrected_p_value)

# Mann-Whitney U Test
U_stat, p_value = stats.mannwhitneyu(faces_data_mean, par_data_mean)

print('Mann-Whitney U test result:')
print(f'U-statistic: {U_stat}')
print(f'p-value: {p_value}')

U_stat, p_value = stats.mannwhitneyu(obj_data_mean, par_data_mean)

print('Mann-Whitney U test result:')
print(f'U-statistic: {U_stat}')
print(f'p-value: {p_value}')

mne.viz.plot_compare_evokeds(dict(obj=obj_evoked, faces=faces_evoked,par=par_evoked),
                             legend='upper left', show_sensors='upper right', picks=['P3', 'P4','P7','P8'])

: 

In [ ]:
# Define the number of comparisons
num_comparisons = 2

# Mann-Whitney U Test
U_stat, p_value = stats.mannwhitneyu(faces_data_mean, par_data_mean)
bonferroni_p_value = p_value * num_comparisons

print('Mann-Whitney U test result:')
print(f'U-statistic: {U_stat}')
print(f'p-value: {p_value}')
print(f'Bonferroni corrected p-value: {bonferroni_p_value}')

U_stat, p_value = stats.mannwhitneyu(obj_data_mean, par_data_mean)
bonferroni_p_value = p_value * num_comparisons

print('Mann-Whitney U test result:')
print(f'U-statistic: {U_stat}')
print(f'p-value: {p_value}')
print(f'Bonferroni corrected p-value: {bonferroni_p_value}')

: 

In [ ]:
# Analysis of N250

obj_data = obj.get_data(picks=['P7', 'P8'], tmin=.23, tmax=.35)
#print(obj_data)

faces_data = faces.get_data(picks=['P7', 'P8'], tmin=.23, tmax=.35)
#print(faces_data)

par_data = par.get_data(picks=['P7', 'P8'], tmin=.23, tmax=.35)
#print(par_data)

# Averaging over the data

# Objects
obj_data_mean = np.mean(obj_data, axis=2) 
obj_data_mean = np.mean(obj_data_mean, axis=1) 


# Faces
faces_data_mean = np.mean(faces_data, axis=2)
faces_data_mean = np.mean(faces_data_mean, axis=1)


# Pareidolias
par_data_mean = np.mean(par_data, axis=2)
par_data_mean = np.mean(par_data_mean, axis=1)

#t_stat, p_value = stats.ttest_ind(faces_data_mean, par_data_mean)

# Define the number of tests conducted
#num_tests = 2  # In this case, you have performed two independent t-tests

# Apply Bonferroni correction to the p-value
#corrected_p_value = p_value * num_tests

# Print the original and corrected p-values
#print(t_stat)
#print("Original p-value:", p_value)
#print("Corrected p-value (Bonferroni):", corrected_p_value)
# Normality testing

##Faces
ks_test = stats.kstest(faces_data_mean,'norm')
ks_test_statistic, ks_test_pvalue = ks_test
print(f"KS test statistic: {ks_test_statistic}, p-value: {ks_test_pvalue}")

##Parreidolias
ks_test = stats.kstest(par_data_mean, 'norm')
ks_test_statistic, ks_test_pvalue = ks_test
print(f"KS test statistic: {ks_test_statistic}, p-value: {ks_test_pvalue}")

##Objects
ks_test = stats.kstest(obj_data_mean, 'norm')
ks_test_statistic, ks_test_pvalue = ks_test
print(f"KS test statistic: {ks_test_statistic}, p-value: {ks_test_pvalue}")



#t_stat, p_value = stats.ttest_ind(faces_data_mean, par_data_mean)

# Define the number of tests conducted
#num_tests = 2  # In this case, you have performed two independent t-tests

# Apply Bonferroni correction to the p-value
#corrected_p_value = p_value * num_tests

# Print the original and corrected p-values
#print(t_stat)
#print("Original p-value:", p_value)
#print("Corrected p-value (Bonferroni):", corrected_p_value)

# Mann-Whitney U Test
U_stat, p_value = stats.mannwhitneyu(faces_data_mean, par_data_mean)

print('Mann-Whitney U test result:')
print(f'U-statistic: {U_stat}')
print(f'p-value: {p_value}')

U_stat, p_value = stats.mannwhitneyu(obj_data_mean, par_data_mean)

print('Mann-Whitney U test result:')
print(f'U-statistic: {U_stat}')
print(f'p-value: {p_value}')




mne.viz.plot_compare_evokeds(dict(obj=obj_evoked, faces=faces_evoked,par=par_evoked ),
                             legend='upper left', show_sensors='upper right',picks=['P7', 'P8'])

: 

In [ ]:
# Define the number of comparisons
num_comparisons = 2

# Mann-Whitney U Test
U_stat, p_value = stats.mannwhitneyu(faces_data_mean, par_data_mean)
bonferroni_p_value = p_value * num_comparisons

print('Mann-Whitney U test result:')
print(f'U-statistic: {U_stat}')
print(f'p-value: {p_value}')
print(f'Bonferroni corrected p-value: {bonferroni_p_value}')

U_stat, p_value = stats.mannwhitneyu(obj_data_mean, par_data_mean)
bonferroni_p_value = p_value * num_comparisons

print('Mann-Whitney U test result:')
print(f'U-statistic: {U_stat}')
print(f'p-value: {p_value}')
print(f'Bonferroni corrected p-value: {bonferroni_p_value}')

: 

: 